In [24]:
"Created a pipeline, loaded toy data into DuckDB, and viewed load info."
"Used dlt.pipeline and pipeline.run methods."
"Used DuckDB, sql_client and dlt dataset to view tables and query data."

'Used DuckDB, sql_client and dlt dataset to view tables and query data.'

In [1]:
import csv
import dlt

In [3]:
data = []

with open("AirPassengers.csv", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        data.append(row)

In [8]:
pipeline = dlt.pipeline(
    pipeline_name="air_passengers_pipeline",
    destination="duckdb",
    dataset_name="air_data",
)

2025-12-30 14:24:59,333|[WARNING]|15968|22732|dlt|pipeline.py|_state_to_props:1717|The destination dlt.destinations.postgres:None in state differs from destination dlt.destinations.duckdb:duckdb in pipeline and will be ignored


In [9]:
load_info = pipeline.run(
    data,
    table_name="air_passengers"
)

print(load_info)

Pipeline air_passengers_pipeline load step completed in 0.84 seconds
1 load package(s) were loaded to destination duckdb and into dataset air_data
The duckdb destination used duckdb:///c:\Users\HP\Desktop\dlt_test\air_passengers_pipeline.duckdb location to store data
Load package 1767084914.8281038 is LOADED and contains no failed jobs


In [20]:
import duckdb

# Path to the DuckDB file created by dlt
db_path = "air_passengers_pipeline.duckdb"

# Connection to DuckDB
conn = duckdb.connect(db_path)

# Set dataset (schema)
conn.execute("SET search_path = 'air_data'")
print(conn.execute("SHOW TABLES").fetchdf())
print(conn.execute("DESCRIBE air_passengers").fetchdf())

# Preview data
df = conn.execute("SELECT * FROM air_passengers LIMIT 10").fetchdf()
print(df)

                  name
0           _dlt_loads
1  _dlt_pipeline_state
2         _dlt_version
3       air_passengers
    column_name column_type null   key default extra
0         month     VARCHAR  YES  None    None  None
1    passengers     VARCHAR  YES  None    None  None
2  _dlt_load_id     VARCHAR   NO  None    None  None
3       _dlt_id     VARCHAR   NO  None    None  None
     month passengers        _dlt_load_id         _dlt_id
0  1949-01        112  1767084914.8281038  t3ALNBi5Aoj6Uw
1  1949-02        118  1767084914.8281038  Mh++XV3/R12QQg
2  1949-03        132  1767084914.8281038  fHn3j9dSDcMfFg
3  1949-04        129  1767084914.8281038  iKQ3xxPAyveTWw
4  1949-05        121  1767084914.8281038  9teJbM9tvuTDjw
5  1949-06        135  1767084914.8281038  0msSGML+oXVjjg
6  1949-07        148  1767084914.8281038  vhqEF4dCp3GuQw
7  1949-08        148  1767084914.8281038  so8WgYJK2Ud30A
8  1949-09        136  1767084914.8281038  2ZZtG8yNCW5i/Q
9  1949-10        119  1767084914.828103

In [21]:
table = conn.sql("SELECT * FROM air_passengers").df()
table

,month,passengers,_dlt_load_id,_dlt_id
0,1949-01,112,1767084914.8281038,t3ALNBi5Aoj6Uw
1,1949-02,118,1767084914.8281038,Mh++XV3/R12QQg
2,1949-03,132,1767084914.8281038,fHn3j9dSDcMfFg
3,1949-04,129,1767084914.8281038,iKQ3xxPAyveTWw
4,1949-05,121,1767084914.8281038,9teJbM9tvuTDjw
...,...,...,...,...
139,1960-08,606,1767084914.8281038,PJzzkP5j20Areg
140,1960-09,508,1767084914.8281038,nqiCgSZrak9hIA
141,1960-10,461,1767084914.8281038,+cLjEVcyO6xCBQ
142,1960-11,390,1767084914.8281038,zzO/uF7t2ldTKA


In [22]:
with pipeline.sql_client() as client:
    with client.execute_query("SELECT * FROM air_passengers") as cursor:
        data = cursor.df()
data

,month,passengers,_dlt_load_id,_dlt_id
0,1949-01,112,1767084914.8281038,t3ALNBi5Aoj6Uw
1,1949-02,118,1767084914.8281038,Mh++XV3/R12QQg
2,1949-03,132,1767084914.8281038,fHn3j9dSDcMfFg
3,1949-04,129,1767084914.8281038,iKQ3xxPAyveTWw
4,1949-05,121,1767084914.8281038,9teJbM9tvuTDjw
...,...,...,...,...
139,1960-08,606,1767084914.8281038,PJzzkP5j20Areg
140,1960-09,508,1767084914.8281038,nqiCgSZrak9hIA
141,1960-10,461,1767084914.8281038,+cLjEVcyO6xCBQ
142,1960-11,390,1767084914.8281038,zzO/uF7t2ldTKA


In [13]:
with pipeline.sql_client() as client:
    with client.execute_query("SELECT Passengers FROM air_passengers") as cursor:
        print(cursor.df())

    passengers
0          112
1          118
2          132
3          129
4          121
..         ...
139        606
140        508
141        461
142        390
143        432

[144 rows x 1 columns]


In [14]:
with pipeline.sql_client() as client:
    with client.execute_query('SELECT "Month", "Passengers" FROM air_passengers') as cursor:
        print(cursor.df())

       month passengers
0    1949-01        112
1    1949-02        118
2    1949-03        132
3    1949-04        129
4    1949-05        121
..       ...        ...
139  1960-08        606
140  1960-09        508
141  1960-10        461
142  1960-11        390
143  1960-12        432

[144 rows x 2 columns]


In [15]:
with pipeline.sql_client() as client:
    with client.execute_query("SHOW TABLES") as cursor:
        print(cursor.df())

                  name
0           _dlt_loads
1  _dlt_pipeline_state
2         _dlt_version
3       air_passengers


In [16]:
with pipeline.sql_client() as client:
    with client.execute_query("SELECT COUNT(*) AS n FROM air_passengers") as cursor:
        print(cursor.df())

     n
0  144


In [17]:
with pipeline.sql_client() as client:
    with client.execute_query("SELECT * FROM air_passengers") as table:
        air_df = table.df()

air_count = air_df.shape[0]

print(f"Number of air passenger records: {air_count}")

Number of air passenger records: 144


In [18]:
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT "Month", "Passengers" FROM air_passengers LIMIT 5'
    ) as cursor:
        preview = cursor.df()

preview

,month,passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [19]:
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT MIN("Month") AS start, MAX("Month") AS end FROM air_passengers'
    ) as cursor:
        print(cursor.df())

     start      end
0  1949-01  1960-12


In [23]:
dataset = pipeline.dataset()
dataset.air_passengers.df()

,month,passengers,_dlt_load_id,_dlt_id
0,1949-01,112,1767084914.8281038,t3ALNBi5Aoj6Uw
1,1949-02,118,1767084914.8281038,Mh++XV3/R12QQg
2,1949-03,132,1767084914.8281038,fHn3j9dSDcMfFg
3,1949-04,129,1767084914.8281038,iKQ3xxPAyveTWw
4,1949-05,121,1767084914.8281038,9teJbM9tvuTDjw
...,...,...,...,...
139,1960-08,606,1767084914.8281038,PJzzkP5j20Areg
140,1960-09,508,1767084914.8281038,nqiCgSZrak9hIA
141,1960-10,461,1767084914.8281038,+cLjEVcyO6xCBQ
142,1960-11,390,1767084914.8281038,zzO/uF7t2ldTKA


In [25]:
import dlt
from dlt.common.typing import TDataItems, TDataItem

@dlt.resource(table_name="air_new") 
def my_dict_list() -> TDataItems:
    yield data

In [28]:
pipeline.dataset().air_passengers.df()

,month,passengers,_dlt_load_id,_dlt_id
0,1949-01,112,1767084914.8281038,t3ALNBi5Aoj6Uw
1,1949-02,118,1767084914.8281038,Mh++XV3/R12QQg
2,1949-03,132,1767084914.8281038,fHn3j9dSDcMfFg
3,1949-04,129,1767084914.8281038,iKQ3xxPAyveTWw
4,1949-05,121,1767084914.8281038,9teJbM9tvuTDjw
...,...,...,...,...
139,1960-08,606,1767084914.8281038,PJzzkP5j20Areg
140,1960-09,508,1767084914.8281038,nqiCgSZrak9hIA
141,1960-10,461,1767084914.8281038,+cLjEVcyO6xCBQ
142,1960-11,390,1767084914.8281038,zzO/uF7t2ldTKA
